In [2]:
pip install flask

In [3]:
pip install --upgrade pyspark

  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Created wheel for pyspark: filename=pyspark-3.2.3-py2.py3-none-any.whl size=281990674 sha256=b69eff572bd437009e0a8c5c98e91a768adf988fdfd2bd42402d8d8438d27dc0
  Stored in directory: c:\users\mcrm1\appdata\local\pip\cache\wheels\0d\a8\47\5bfb7504570634f4aca1407ef83a55705ff7730612a5f6f8b7
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests

In [5]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
from xgboost import XGBClassifier

In [7]:
from urllib.parse import urlparse,urlencode
import ipaddress
import re

In [8]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__)
model = pickle.load(open('XGBoostClassifier.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    url = request.form.get("experience")
    #data = request.get_json(force=True)
    #output = [45,6,8]
    #feature functions

    #address
    def getDomain(url):  
        domain = urlparse(url).netloc
        if re.match(r"^www.",domain):
	        domain = domain.replace("www.","")
        return domain
    
    def havingIP(url):
        try:
            ipaddress.ip_address(url)
            ip = 1
        except:
            ip = 0
        return ip

    def haveAtSign(url):
        if "@" in url:
            at = 1    
        else:
            at = 0    
        return at

    def getLength(url):
        if len(url) < 54:
            length = 0            
        else:
            length = 1            
        return length

    def getDepth(url):
        s = urlparse(url).path.split('/')
        depth = 0
        for j in range(len(s)):
            if len(s[j]) != 0:
                depth = depth+1
        return depth

    def redirection(url):
        pos = url.rfind('//')
        if pos > 6:
            if pos > 7:
                return 1
            else:
                return 0
        else:
            return 0

    def httpDomain(url):
        domain = urlparse(url).netloc
        if 'https' in domain:
            return 1
        else:
            return 0

    shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                        r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                        r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                        r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                        r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                        r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                        r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                        r"tr\.im|link\.zip\.net"
    
    def tinyURL(url):
        match=re.search(shortening_services,url)
        if match:
            return 1
        else:
            return 0
        
    def prefixSuffix(url):
        if '-' in urlparse(url).netloc:
            return 1            # phishing
        else:
            return 0            # legitimate

    #domain
    
    
    #html js
    def iframe(response):
        if response == "":
            return 1
        else:
            if re.findall(r"[<iframe>|<frameBorder>]", response.text):
                return 0
            else:
                return 1
    
    def mouseOver(response): 
        if response == "" :
            return 1
        else:
            if re.findall("<script>.+onmouseover.+</script>", response.text):
                return 1
            else:
                return 0
    
    def rightClick(response):
        if response == "":
            return 1
        else:
            if re.findall(r"event.button ?== ?2", response.text):
                return 0
            else:
                return 1   

    def forwarding(response):
        if response == "":
            return 1
        else:
            if len(response.history) <= 2:
                return 0
            else:
                return 1
            
    
    



    #feature creation
    try:
        response = requests.get(url)
    except:
        response = ""
    output=[]
    output.append(getDomain(url))
    output.append(havingIP(url))
    output.append(haveAtSign(url))
    output.append(getLength(url))
    output.append(getDepth(url))
    output.append(redirection(url))
    output.append(httpDomain(url))
    output.append(tinyURL(url))
    output.append(prefixSuffix(url))
    output.append(iframe(response))
    output.append(mouseOver(response))
    output.append(rightClick(response))
    output.append(forwarding(response))
    return render_template('index.html', prediction_text='the website is  {}'.format(output))

@app.route('/predict_api',methods=['POST'])
def predict_api():
    '''
    For direct API calls trought request
    '''
    

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)
    

FileNotFoundError: [Errno 2] No such file or directory: 'XGBoostClassifier.pkl'